In [22]:
!pip3 install pulp


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd

df = pd.read_csv("data/player_stats.csv")
df = df[df["yearID"] == 2014]
df = df[df["leagueID"] == "NL"]
# filter where league is NL
df = df[["playerID", "br_WAR_total", "salary"]]
# get rows where no column has NaN
df = df.dropna()
# keep unique playerID and keep the first if duplicates are found
df = df.drop_duplicates(subset="playerID", keep="first")

In [24]:
df

,playerID,br_WAR_total,salary
2870,blackvi01,0.35,502250.0
2871,brownan02,-0.09,538045.0
2875,colonba01,-0.14,9000000.0
2876,darnatr01,0.49,501560.0
2878,dudalu01,3.76,1637500.0
...,...,...,...
39375,schaflo01,-0.26,505000.0
39376,segurje01,0.89,534000.0
39377,smithwi04,0.06,502000.0
39378,thornty01,0.06,505000.0


In [26]:
import pulp

df1 = df

budget = 100000000

# Create a linear programming model
model = pulp.LpProblem("Best Baseball Team", pulp.LpMaximize)

# Define the variables for the problem
# Each variable represents a player and has a value of 1 if the player is selected for the team and 0 otherwise
player_vars = pulp.LpVariable.dicts("Player", df1["playerID"].unique().tolist(), 0, 1, pulp.LpInteger)

# Define the objective function
# The objective function is to maximize the total WAR of the team
model += sum(player_vars[row["playerID"]] * row["br_WAR_total"] for index, row in df1.iterrows())

# Define the budget constraint
# The total salary of the team must be less than or equal to the budget
model += sum(player_vars[row["playerID"]] * row["salary"] for index, row in df1.iterrows()) <= budget

# Define the position constraints
# The number of players at each position must be within the allowed range
model += sum(player_vars[row["playerID"]] for index, row in df1.iterrows()) == 26

print(model)

# Solve the optimization problem
model.solve()

/Users/oliverjarvis/.pyenv/versions/anaconda3-sd/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Best_Baseball_Team:
MAXIMIZE
-0.12*Player_abreuto01 + 2.23*Player_adamsma01 + 0.22*Player_adamsmi03 + -0.48*Player_adriaeh01 + 1.26*Player_affelje01 + 1.3*Player_alonsyo01 + 4.84*Player_alvarhe01 + -0.64*Player_alvarpe01 + 1.77*Player_amarial01 + 1.05*Player_anderbr04 + 3.63*Player_arenano01 + -0.4*Player_ariasjo01 + 5.609999999999999*Player_arrieja01 + 0.75*Player_arroybr01 + 0.3*Player_ascheco01 + -0.63*Player_aumonph01 + 1.51*Player_baileho02 + 0.04*Player_bakerje03 + -1.22*Player_bakerjo01 + 0.7*Player_barmecl01 + 0.44*Player_barnebr02 + 0.11*Player_bastaan01 + -0.14*Player_baxtemi01 + 2.33*Player_beckejo02 + 0.15*Player_belisma01 + 0.96*Player_beltbr01 + -1.14*Player_bettich01 + -0.42*Player_biancje01 + 2.01*Player_blackch02 + 0.35*Player_blackvi01 + -0.4*Player_blevije01 + 1.41*Player_bourjpe01 + 1.17*Player_braunry02 + -0.9*Player_brothre01 + -0.09*Player_brownan02 + -1.3*Player_browndo01 + -0.84*Player_bruceja01 + 4.890000000000001*Player_bumgama01 + -0.31*Player_buterdr01 + -0

1

In [ ]:
df1

,playerID,br_WAR_total,salary
2870,blackvi01,0.35,502250.0
2871,brownan02,-0.09,538045.0
2875,colonba01,-0.14,9000000.0
2876,darnatr01,0.49,501560.0
2878,dudalu01,3.76,1637500.0
...,...,...,...
39375,schaflo01,-0.26,505000.0
39376,segurje01,0.89,534000.0
39377,smithwi04,0.06,502000.0
39378,thornty01,0.06,505000.0


In [30]:
total_war = 0
for var in model.variables():
    if var.varValue == 1:
        
        #find the row where playerID is equal to the variable name
       total_war += df1[df1["playerID"] == var.name[7:]]["br_WAR_total"].values[0]
print(total_war)

140.56
